In [1]:
import pandas as pd
import numpy as np
import string
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer

In [2]:
train = pd.read_csv('trainMaster.csv')

# Target

In [3]:
x = train['comment_text']

y1 = train['toxic']
y2 = train['severe_toxic']
y3 = train['obscene']
y4 = train['threat']
y5 = train['insult']
y6 = train['identity_hate']
y7 = train['clean']

datay = [y1, y2, y3, y4, y5, y6, y7]

# Train Test Split

In [4]:
from sklearn.model_selection import train_test_split

In [5]:
dSplit = {}
for j in range(1,8):
        X_train, X_test, dSplit['y{}_train'.format(j)], dSplit['y{}_test'.format(j)] = train_test_split(x, datay[j-1], test_size=0.3, random_state=101)

In [6]:
print('OwO')

OwO


# K Fold

In [7]:
# from sklearn.model_selection import KFold

In [8]:
# K = 5
# kf = KFold(n_splits=K, shuffle=True, random_state = 42)

In [9]:
# dFold = {}
# for i in range(1,8):
#     dFold['Hasilfold{}'.format(i)] = kf.split(x,datay[i-1])

# Model

In [10]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from xgboost import XGBClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer


# Analyzer for Pipeline

In [11]:
stop = []
for words in stopwords.words('english'):
    s = [char for char in words if char not in string.punctuation]
    stop.append(''.join(s))
stop.extend(['may','also','across','among','beside','however','yet','within'])

# stemmer = SnowballStemmer("english")
# def stemming(sentence):
#     stemSentence = ""
#     for word in sentence.split():
#         stem = stemmer.stem(word)
#         stemSentence += stem
#         stemSentence += " "
#     stemSentence = stemSentence.strip()
#     return stemSentence

def process_normal(text):
    return [word for word in text.split() if word.lower() not in stop]

def process_nocase(text):
    return [word.lower() for word in text.split() if word.lower() not in stop]

def process_nopunc(text):
    nopunc = [char for char in text if char not in string.punctuation]
    nopunc = ''.join(nopunc)
    return [word for word in nopunc.split() if word.lower() not in stop]

def process_nocasepunc(text):
    nocasepunc = [char for char in text if char not in string.punctuation]
    nocasepunc = ''.join(nocasepunc)
    return [word.lower() for word in nocasepunc.split() if word.lower() not in stop]

# Pipeline

In [12]:
listAnalyzer = [process_normal, process_nocase,process_nopunc,process_nocasepunc]
nama = ['normal','noCase','noPunc','noCasePunc']
dPP = {}
dPP2 = {}

# buat analyzer
for i in range(0,4): 
#     buat n_gram
    for j in range(1,4):
        dPP['pp_{}_ngram_{}'.format(nama[i],j)] = Pipeline([
            ('bow', CountVectorizer(analyzer=listAnalyzer[i], ngram_range=(1,j), lowercase=False)),
            ('classifier', XGBClassifier()),
        ])

        # With TFIDF
        dPP2['pp2_{}_ngram_{}'.format(nama[i],j)] = Pipeline([
            ('bow', CountVectorizer(analyzer=listAnalyzer[i], ngram_range=(1,j), lowercase=False)),
            ('tfidf', TfidfTransformer()),
            ('classifier', XGBClassifier()),
        ])

In [13]:
print('OwO')

OwO


# Predict

In [14]:
ppKey = list(dPP.keys())
pp2Key = list(dPP2.keys())

dPredict_train = {}
dPredict_test = {}
dPredictProba_train = {}
dPredictProba_test = {}

dPredict2_train = {}
dPredict2_test = {}
dPredict2Proba_train = {}
dPredict2Proba_test = {}

for item, item2 in zip(ppKey, pp2Key):
    for i in range(1,8):
#         Pipeline
        dPP[item].fit(X_train, dSplit['y{}_train'.format(i)])
        
        dPredict_train['predictions_train_y{}_{}'.format(i,item)] = dPP[item].predict(X_train)
        dPredictProba_train['predictions_train_y{}_{}'.format(i,item)] = dPP[item].predict_proba(X_train)
        
        dPredict_test['predictions_test_y{}_{}'.format(i,item)] = dPP[item].predict(X_test)
        dPredictProba_test['predictions_test_y{}_{}'.format(i,item)] = dPP[item].predict_proba(X_test)
        
#         Pipeline2
        dPP2[item2].fit(X_train, dSplit['y{}_train'.format(i)])
    
        dPredict2_train['predictions2_train_y{}_{}'.format(i,item2)] = dPP2[item2].predict(X_train)
        dPredict2Proba_train['predictions2_train_y{}_{}'.format(i,item2)] = dPP2[item2].predict_proba(X_train)
        
        dPredict2_test['predictions2_test_y{}_{}'.format(i,item2)] = dPP2[item2].predict(X_test)
        dPredict2Proba_test['predictions2_test_y{}_{}'.format(i,item2)] = dPP2[item2].predict_proba(X_test)

In [15]:
print('OwO')

OwO


# Report

In [16]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, roc_auc_score, matthews_corrcoef, f1_score, log_loss

In [17]:
def calc_train_error(X_train, y_train, predictTrain, predictProbaTrain):
    matt = matthews_corrcoef(y_train, predictTrain)
    f1 = f1_score(y_train, predictTrain, average = 'macro')
    report = classification_report(y_train, predictTrain)
    roc_auc = roc_auc_score(y_train, predictProbaTrain[:,1])
    accuracy = accuracy_score(y_train, predictTrain)
    confMatrix = confusion_matrix(y_train, predictTrain)
    logloss = log_loss(y_train, predictProbaTrain)
    return {
        'report':report,
        'matthew':matt,
        'f1':f1,
        'roc':roc_auc,
        'accuracy':accuracy,
        'confusion':confMatrix,
        'logloss':logloss
    }

def calc_validation_error(X_test, y_test, predictTest, predictProbaTest):
    matt = matthews_corrcoef(y_test, predictTest)
    f1 = f1_score(y_test, predictTest, average = 'macro')
    report = classification_report(y_test, predictTest)
    roc_auc = roc_auc_score(y_test, predictProbaTest[:,1])
    accuracy = accuracy_score(y_test, predictTest)
    confMatrix = confusion_matrix(y_test, predictTest)
    logloss = log_loss(y_test, predictProbaTest)
    return {
        'report':report,
        'matthew':matt,
        'f1':f1,
        'roc':roc_auc,
        'accuracy':accuracy,
        'confusion':confMatrix,
        'logloss':logloss
    }

def calc_metrics(X_train, y_train, X_test, y_test, predictTrain, predictProbaTrain, predictTest, predictProbaTest):
    train_error = calc_train_error(X_train, y_train, predictTrain, predictProbaTrain)
    validation_error = calc_validation_error(X_test, y_test, predictTest, predictProbaTest)
    return train_error, validation_error

## Toxic

In [18]:
dToxic_train = {}
dToxic_test = {}
dToxic2_train = {}
dToxic2_test = {}

for item in ppKey:
    dToxic_train['hasil_train_{}'.format(item)], dToxic_test['hasil_test_{}'.format(item)]= calc_metrics(X_train, dSplit['y1_train'], X_test, dSplit['y1_test'], 
                                                                                                          dPredict_train['predictions_train_y1_{}'.format(item)],
                                                                                                          dPredictProba_train['predictions_train_y1_{}'.format(item)],
                                                                                                          dPredict_test['predictions_test_y1_{}'.format(item)],
                                                                                                          dPredictProba_test['predictions_test_y1_{}'.format(item)])

for item2 in pp2Key:
    dToxic2_train['hasil_train_{}'.format(item2)], dToxic2_test['hasil_test_{}'.format(item2)]= calc_metrics(X_train, dSplit['y1_train'], X_test, dSplit['y1_test'], 
                                                                                                          dPredict2_train['predictions2_train_y1_{}'.format(item2)],
                                                                                                          dPredict2Proba_train['predictions2_train_y1_{}'.format(item2)],
                                                                                                          dPredict2_test['predictions2_test_y1_{}'.format(item2)],
                                                                                                          dPredict2Proba_test['predictions2_test_y1_{}'.format(item2)])

In [19]:
dToxic_train_key = list(dToxic_train.keys())
dToxic_test_key = list(dToxic_test.keys())
listItem = []

for train, test in zip(dToxic_train_key, dToxic_test_key):
    listItem.append(['xgb_{}'.format(test),
                     dToxic_train[train]['accuracy'],
                     dToxic_test[test]['accuracy'],
                     dToxic_train[train]['roc'],
                     dToxic_test[test]['roc'],
                     dToxic_train[train]['f1'],
                     dToxic_test[test]['f1'],
                     dToxic_train[train]['matthew'],
                     dToxic_test[test]['matthew'],
                     dToxic_train[train]['logloss'],
                     dToxic_test[test]['logloss'],])


dfToxicResult = pd.DataFrame(columns=['Model','Train Accuracy', 'Test Accuracy', 'Train ROC AUC', 'Test ROC AUC', 'Train F1 Score','Test F1 Score', 
                                      'Train Matthews Corr Coef', 'Test Matthew Corr Coef','Train Log Loss','Test Log Loss'], data=listItem)
dfToxicResult

,Model,Train Accuracy,Test Accuracy,Train ROC AUC,Test ROC AUC,Train F1 Score,Test F1 Score,Train Matthews Corr Coef,Test Matthew Corr Coef,Train Log Loss,Test Log Loss
0,xgb_hasil_test_pp_normal_ngram_1,0.928630,0.928434,0.865376,0.863316,0.687655,0.691571,0.484076,0.486326,0.219438,0.221082
1,xgb_hasil_test_pp_normal_ngram_2,0.928630,0.928434,0.865376,0.863316,0.687655,0.691571,0.484076,0.486326,0.219438,0.221082
2,xgb_hasil_test_pp_normal_ngram_3,0.928630,0.928434,0.865376,0.863316,0.687655,0.691571,0.484076,0.486326,0.219438,0.221082
3,xgb_hasil_test_pp_noCase_ngram_1,0.932148,0.931881,0.882266,0.879867,0.712202,0.715342,0.518830,0.520100,0.206646,0.207673
4,xgb_hasil_test_pp_noCase_ngram_2,0.932148,0.931881,0.882266,0.879867,0.712202,0.715342,0.518830,0.520100,0.206646,0.207673
5,xgb_hasil_test_pp_noCase_ngram_3,0.932148,0.931881,0.882266,0.879867,0.712202,0.715342,0.518830,0.520100,0.206646,0.207673
6,xgb_hasil_test_pp_noPunc_ngram_1,0.931405,0.930711,0.890390,0.889072,0.707945,0.708321,0.511554,0.508796,0.206302,0.207540
7,xgb_hasil_test_pp_noPunc_ngram_2,0.931405,0.930711,0.890390,0.889072,0.707945,0.708321,0.511554,0.508796,0.206302,0.207540
8,xgb_hasil_test_pp_noPunc_ngram_3,0.931405,0.930711,0.890390,0.889072,0.707945,0.708321,0.511554,0.508796,0.206302,0.207540
9,xgb_hasil_test_pp_noCasePunc_ngram_1,0.935084,0.934784,0.906864,0.906161,0.732765,0.735395,0.546255,0.547127,0.190909,0.191811


In [20]:
dToxic2_train_key = list(dToxic2_train.keys())
dToxic2_test_key = list(dToxic2_test.keys())
listItem = []

for train, test in zip(dToxic2_train_key, dToxic2_test_key):
    listItem.append(['xgb_{}'.format(test),
                     dToxic2_train[train]['accuracy'],
                     dToxic2_test[test]['accuracy'],
                     dToxic2_train[train]['roc'],
                     dToxic2_test[test]['roc'],
                     dToxic2_train[train]['f1'],
                     dToxic2_test[test]['f1'],
                     dToxic2_train[train]['matthew'],
                     dToxic2_test[test]['matthew'],
                     dToxic2_train[train]['logloss'],
                     dToxic2_test[test]['logloss'],])


dfToxic2Result = pd.DataFrame(columns=['Model','Train Accuracy', 'Test Accuracy', 'Train ROC AUC', 'Test ROC AUC', 'Train F1 Score','Test F1 Score', 
                                      'Train Matthews Corr Coef', 'Test Matthew Corr Coef','Train Log Loss','Test Log Loss'], data=listItem)
dfToxic2Result

,Model,Train Accuracy,Test Accuracy,Train ROC AUC,Test ROC AUC,Train F1 Score,Test F1 Score,Train Matthews Corr Coef,Test Matthew Corr Coef,Train Log Loss,Test Log Loss
0,xgb_hasil_test_pp2_normal_ngram_1,0.928728,0.928956,0.867536,0.863041,0.686943,0.694449,0.485362,0.491665,0.217526,0.219082
1,xgb_hasil_test_pp2_normal_ngram_2,0.928728,0.928956,0.867536,0.863041,0.686943,0.694449,0.485362,0.491665,0.217526,0.219082
2,xgb_hasil_test_pp2_normal_ngram_3,0.928728,0.928956,0.867536,0.863041,0.686943,0.694449,0.485362,0.491665,0.217526,0.219082
3,xgb_hasil_test_pp2_noCase_ngram_1,0.933733,0.933636,0.882527,0.879568,0.720611,0.725184,0.534193,0.536728,0.203650,0.204673
4,xgb_hasil_test_pp2_noCase_ngram_2,0.933733,0.933636,0.882527,0.879568,0.720611,0.725184,0.534193,0.536728,0.203650,0.204673
5,xgb_hasil_test_pp2_noCase_ngram_3,0.933733,0.933636,0.882527,0.879568,0.720611,0.725184,0.534193,0.536728,0.203650,0.204673
6,xgb_hasil_test_pp2_noPunc_ngram_1,0.933106,0.933134,0.889721,0.887751,0.716886,0.722187,0.528229,0.532040,0.203154,0.204352
7,xgb_hasil_test_pp2_noPunc_ngram_2,0.933106,0.933134,0.889721,0.887751,0.716886,0.722187,0.528229,0.532040,0.203154,0.204352
8,xgb_hasil_test_pp2_noPunc_ngram_3,0.933106,0.933134,0.889721,0.887751,0.716886,0.722187,0.528229,0.532040,0.203154,0.204352
9,xgb_hasil_test_pp2_noCasePunc_ngram_1,0.938415,0.938691,0.907137,0.905046,0.750627,0.756711,0.576434,0.581912,0.186239,0.186906


In [21]:
# print(confusion_matrix(dSplit['y1_test'], predictions))
# print('\n')
# print(classification_report(dSplit['y1_test'], predictions))

## Severe Toxic

In [22]:
dSToxic_train = {}
dSToxic_test = {}
dSToxic2_train = {}
dSToxic2_test = {}

for item in ppKey:
    dSToxic_train['hasil_train_{}'.format(item)], dSToxic_test['hasil_test_{}'.format(item)]= calc_metrics(X_train, dSplit['y2_train'], X_test, dSplit['y2_test'], 
                                                                                                          dPredict_train['predictions_train_y2_{}'.format(item)],
                                                                                                          dPredictProba_train['predictions_train_y2_{}'.format(item)],
                                                                                                          dPredict_test['predictions_test_y2_{}'.format(item)],
                                                                                                          dPredictProba_test['predictions_test_y2_{}'.format(item)])

for item2 in pp2Key:
    dSToxic2_train['hasil_train_{}'.format(item2)], dSToxic2_test['hasil_test_{}'.format(item2)]= calc_metrics(X_train, dSplit['y2_train'], X_test, dSplit['y2_test'], 
                                                                                                          dPredict2_train['predictions2_train_y2_{}'.format(item2)],
                                                                                                          dPredict2Proba_train['predictions2_train_y2_{}'.format(item2)],
                                                                                                          dPredict2_test['predictions2_test_y2_{}'.format(item2)],
                                                                                                          dPredict2Proba_test['predictions2_test_y2_{}'.format(item2)])

In [23]:
dSToxic_train_key = list(dSToxic_train.keys())
dSToxic_test_key = list(dSToxic_test.keys())
listItem = []

for train, test in zip(dSToxic_train_key, dSToxic_test_key):
    listItem.append(['xgb_{}'.format(test),
                     dSToxic_train[train]['accuracy'],
                     dSToxic_test[test]['accuracy'],
                     dSToxic_train[train]['roc'],
                     dSToxic_test[test]['roc'],
                     dSToxic_train[train]['f1'],
                     dSToxic_test[test]['f1'],
                     dSToxic_train[train]['matthew'],
                     dSToxic_test[test]['matthew'],
                     dSToxic_train[train]['logloss'],
                     dSToxic_test[test]['logloss'],])


dfSToxicResult = pd.DataFrame(columns=['Model','Train Accuracy', 'Test Accuracy', 'Train ROC AUC', 'Test ROC AUC', 'Train F1 Score','Test F1 Score', 
                                      'Train Matthews Corr Coef', 'Test Matthew Corr Coef','Train Log Loss','Test Log Loss'], data=listItem)
dfSToxicResult

,Model,Train Accuracy,Test Accuracy,Train ROC AUC,Test ROC AUC,Train F1 Score,Test F1 Score,Train Matthews Corr Coef,Test Matthew Corr Coef,Train Log Loss,Test Log Loss
0,xgb_hasil_test_pp_normal_ngram_1,0.990913,0.989597,0.959038,0.946514,0.587449,0.575349,0.276924,0.216863,0.032434,0.035748
1,xgb_hasil_test_pp_normal_ngram_2,0.990913,0.989597,0.959038,0.946514,0.587449,0.575349,0.276924,0.216863,0.032434,0.035748
2,xgb_hasil_test_pp_normal_ngram_3,0.990913,0.989597,0.959038,0.946514,0.587449,0.575349,0.276924,0.216863,0.032434,0.035748
3,xgb_hasil_test_pp_noCase_ngram_1,0.991164,0.989493,0.966682,0.959344,0.606422,0.556902,0.316854,0.181059,0.030023,0.033032
4,xgb_hasil_test_pp_noCase_ngram_2,0.991164,0.989493,0.966682,0.959344,0.606422,0.556902,0.316854,0.181059,0.030023,0.033032
5,xgb_hasil_test_pp_noCase_ngram_3,0.991164,0.989493,0.966682,0.959344,0.606422,0.556902,0.316854,0.181059,0.030023,0.033032
6,xgb_hasil_test_pp_noPunc_ngram_1,0.990949,0.989576,0.967281,0.953868,0.591700,0.575212,0.284166,0.215579,0.031460,0.034551
7,xgb_hasil_test_pp_noPunc_ngram_2,0.990949,0.989576,0.967281,0.953868,0.591700,0.575212,0.284166,0.215579,0.031460,0.034551
8,xgb_hasil_test_pp_noPunc_ngram_3,0.990949,0.989576,0.967281,0.953868,0.591700,0.575212,0.284166,0.215579,0.031460,0.034551
9,xgb_hasil_test_pp_noCasePunc_ngram_1,0.990985,0.989660,0.975073,0.970070,0.601589,0.577195,0.296293,0.223007,0.028785,0.031714


In [24]:
dSToxic2_train_key = list(dSToxic2_train.keys())
dSToxic2_test_key = list(dSToxic2_test.keys())
listItem = []

for train, test in zip(dSToxic2_train_key, dSToxic2_test_key):
    listItem.append(['xgb_{}'.format(test),
                     dSToxic2_train[train]['accuracy'],
                     dSToxic2_test[test]['accuracy'],
                     dSToxic2_train[train]['roc'],
                     dSToxic2_test[test]['roc'],
                     dSToxic2_train[train]['f1'],
                     dSToxic2_test[test]['f1'],
                     dSToxic2_train[train]['matthew'],
                     dSToxic2_test[test]['matthew'],
                     dSToxic2_train[train]['logloss'],
                     dSToxic2_test[test]['logloss'],])


dfSToxic2Result = pd.DataFrame(columns=['Model','Train Accuracy', 'Test Accuracy', 'Train ROC AUC', 'Test ROC AUC', 'Train F1 Score','Test F1 Score', 
                                      'Train Matthews Corr Coef', 'Test Matthew Corr Coef','Train Log Loss','Test Log Loss'], data=listItem)
dfSToxic2Result

,Model,Train Accuracy,Test Accuracy,Train ROC AUC,Test ROC AUC,Train F1 Score,Test F1 Score,Train Matthews Corr Coef,Test Matthew Corr Coef,Train Log Loss,Test Log Loss
0,xgb_hasil_test_pp2_normal_ngram_1,0.991164,0.989535,0.956645,0.944958,0.597535,0.567694,0.311672,0.201426,0.031291,0.034777
1,xgb_hasil_test_pp2_normal_ngram_2,0.991164,0.989535,0.956645,0.944958,0.597535,0.567694,0.311672,0.201426,0.031291,0.034777
2,xgb_hasil_test_pp2_normal_ngram_3,0.991164,0.989535,0.956645,0.944958,0.597535,0.567694,0.311672,0.201426,0.031291,0.034777
3,xgb_hasil_test_pp2_noCase_ngram_1,0.991271,0.989911,0.966284,0.958329,0.612430,0.587445,0.331595,0.252034,0.028779,0.032465
4,xgb_hasil_test_pp2_noCase_ngram_2,0.991271,0.989911,0.966284,0.958329,0.612430,0.587445,0.331595,0.252034,0.028779,0.032465
5,xgb_hasil_test_pp2_noCase_ngram_3,0.991271,0.989911,0.966284,0.958329,0.612430,0.587445,0.331595,0.252034,0.028779,0.032465
6,xgb_hasil_test_pp2_noPunc_ngram_1,0.991101,0.989806,0.968737,0.957967,0.603943,0.592120,0.308896,0.252358,0.029983,0.033707
7,xgb_hasil_test_pp2_noPunc_ngram_2,0.991101,0.989806,0.968737,0.957967,0.603943,0.592120,0.308896,0.252358,0.029983,0.033707
8,xgb_hasil_test_pp2_noPunc_ngram_3,0.991101,0.989806,0.968737,0.957967,0.603943,0.592120,0.308896,0.252358,0.029983,0.033707
9,xgb_hasil_test_pp2_noCasePunc_ngram_1,0.991459,0.989994,0.974900,0.971462,0.629228,0.604215,0.359844,0.278449,0.027314,0.030920


## Obscene

In [25]:
dObscene_train = {}
dObscene_test = {}
dObscene2_train = {}
dObscene2_test = {}

for item in ppKey:
    dObscene_train['hasil_train_{}'.format(item)], dObscene_test['hasil_test_{}'.format(item)]= calc_metrics(X_train, dSplit['y3_train'], X_test, dSplit['y3_test'], 
                                                                                                          dPredict_train['predictions_train_y3_{}'.format(item)],
                                                                                                          dPredictProba_train['predictions_train_y3_{}'.format(item)],
                                                                                                          dPredict_test['predictions_test_y3_{}'.format(item)],
                                                                                                          dPredictProba_test['predictions_test_y3_{}'.format(item)])

for item2 in pp2Key:
    dObscene2_train['hasil_train_{}'.format(item2)], dObscene2_test['hasil_test_{}'.format(item2)]= calc_metrics(X_train, dSplit['y3_train'], X_test, dSplit['y3_test'], 
                                                                                                          dPredict2_train['predictions2_train_y3_{}'.format(item2)],
                                                                                                          dPredict2Proba_train['predictions2_train_y3_{}'.format(item2)],
                                                                                                          dPredict2_test['predictions2_test_y3_{}'.format(item2)],
                                                                                                          dPredict2Proba_test['predictions2_test_y3_{}'.format(item2)])

In [26]:
dObscene_train_key = list(dObscene_train.keys())
dObscene_test_key = list(dObscene_test.keys())
listItem = []

for train, test in zip(dObscene_train_key, dObscene_test_key):
    listItem.append(['xgb_{}'.format(test),
                     dObscene_train[train]['accuracy'],
                     dObscene_test[test]['accuracy'],
                     dObscene_train[train]['roc'],
                     dObscene_test[test]['roc'],
                     dObscene_train[train]['f1'],
                     dObscene_test[test]['f1'],
                     dObscene_train[train]['matthew'],
                     dObscene_test[test]['matthew'],
                     dObscene_train[train]['logloss'],
                     dObscene_test[test]['logloss'],])


dfObsceneResult = pd.DataFrame(columns=['Model','Train Accuracy', 'Test Accuracy', 'Train ROC AUC', 'Test ROC AUC', 'Train F1 Score','Test F1 Score', 
                                      'Train Matthews Corr Coef', 'Test Matthew Corr Coef','Train Log Loss','Test Log Loss'], data=listItem)
dfObsceneResult

,Model,Train Accuracy,Test Accuracy,Train ROC AUC,Test ROC AUC,Train F1 Score,Test F1 Score,Train Matthews Corr Coef,Test Matthew Corr Coef,Train Log Loss,Test Log Loss
0,xgb_hasil_test_pp_normal_ngram_1,0.968487,0.967455,0.913297,0.903606,0.785333,0.785118,0.622734,0.618162,0.113779,0.117484
1,xgb_hasil_test_pp_normal_ngram_2,0.968487,0.967455,0.913297,0.903606,0.785333,0.785118,0.622734,0.618162,0.113779,0.117484
2,xgb_hasil_test_pp_normal_ngram_3,0.968487,0.967455,0.913297,0.903606,0.785333,0.785118,0.622734,0.618162,0.113779,0.117484
3,xgb_hasil_test_pp_noCase_ngram_1,0.970036,0.969481,0.933293,0.926465,0.801750,0.804787,0.646052,0.647968,0.102852,0.106238
4,xgb_hasil_test_pp_noCase_ngram_2,0.970036,0.969481,0.933293,0.926465,0.801750,0.804787,0.646052,0.647968,0.102852,0.106238
5,xgb_hasil_test_pp_noCase_ngram_3,0.970036,0.969481,0.933293,0.926465,0.801750,0.804787,0.646052,0.647968,0.102852,0.106238
6,xgb_hasil_test_pp_noPunc_ngram_1,0.970107,0.968917,0.938751,0.933341,0.802483,0.800247,0.647114,0.640038,0.100235,0.104248
7,xgb_hasil_test_pp_noPunc_ngram_2,0.970107,0.968917,0.938751,0.933341,0.802483,0.800247,0.647114,0.640038,0.100235,0.104248
8,xgb_hasil_test_pp_noPunc_ngram_3,0.970107,0.968917,0.938751,0.933341,0.802483,0.800247,0.647114,0.640038,0.100235,0.104248
9,xgb_hasil_test_pp_noCasePunc_ngram_1,0.973124,0.972155,0.959854,0.953510,0.830194,0.829331,0.690167,0.685731,0.087135,0.090946


In [27]:
dObscene2_train_key = list(dObscene2_train.keys())
dObscene2_test_key = list(dObscene2_test.keys())
listItem = []

for train, test in zip(dObscene2_train_key, dObscene2_test_key):
    listItem.append(['xgb_{}'.format(test),
                     dObscene2_train[train]['accuracy'],
                     dObscene2_test[test]['accuracy'],
                     dObscene2_train[train]['roc'],
                     dObscene2_test[test]['roc'],
                     dObscene2_train[train]['f1'],
                     dObscene2_test[test]['f1'],
                     dObscene2_train[train]['matthew'],
                     dObscene2_test[test]['matthew'],
                     dObscene2_train[train]['logloss'],
                     dObscene2_test[test]['logloss'],])


dfObscene2Result = pd.DataFrame(columns=['Model','Train Accuracy', 'Test Accuracy', 'Train ROC AUC', 'Test ROC AUC', 'Train F1 Score','Test F1 Score', 
                                      'Train Matthews Corr Coef', 'Test Matthew Corr Coef','Train Log Loss','Test Log Loss'], data=listItem)
dfObscene2Result

,Model,Train Accuracy,Test Accuracy,Train ROC AUC,Test ROC AUC,Train F1 Score,Test F1 Score,Train Matthews Corr Coef,Test Matthew Corr Coef,Train Log Loss,Test Log Loss
0,xgb_hasil_test_pp2_normal_ngram_1,0.968621,0.967894,0.913189,0.903427,0.784573,0.787636,0.624567,0.624309,0.112214,0.116184
1,xgb_hasil_test_pp2_normal_ngram_2,0.968621,0.967894,0.913189,0.903427,0.784573,0.787636,0.624567,0.624309,0.112214,0.116184
2,xgb_hasil_test_pp2_normal_ngram_3,0.968621,0.967894,0.913189,0.903427,0.784573,0.787636,0.624567,0.624309,0.112214,0.116184
3,xgb_hasil_test_pp2_noCase_ngram_1,0.971056,0.970421,0.932047,0.925628,0.808878,0.812410,0.660442,0.661061,0.100504,0.103945
4,xgb_hasil_test_pp2_noCase_ngram_2,0.971056,0.970421,0.932047,0.925628,0.808878,0.812410,0.660442,0.661061,0.100504,0.103945
5,xgb_hasil_test_pp2_noCase_ngram_3,0.971056,0.970421,0.932047,0.925628,0.808878,0.812410,0.660442,0.661061,0.100504,0.103945
6,xgb_hasil_test_pp2_noPunc_ngram_1,0.971020,0.970108,0.937527,0.932185,0.808475,0.808983,0.659920,0.656476,0.097894,0.101891
7,xgb_hasil_test_pp2_noPunc_ngram_2,0.971020,0.970108,0.937527,0.932185,0.808475,0.808983,0.659920,0.656476,0.097894,0.101891
8,xgb_hasil_test_pp2_noPunc_ngram_3,0.971020,0.970108,0.937527,0.932185,0.808475,0.808983,0.659920,0.656476,0.097894,0.101891
9,xgb_hasil_test_pp2_noCasePunc_ngram_1,0.973581,0.972761,0.958718,0.952762,0.832014,0.832927,0.695926,0.693385,0.083853,0.087386


## Threat

In [28]:
dThreat_train = {}
dThreat_test = {}
dThreat2_train = {}
dThreat2_test = {}

for item in ppKey:
    dThreat_train['hasil_train_{}'.format(item)], dThreat_test['hasil_test_{}'.format(item)]= calc_metrics(X_train, dSplit['y4_train'], X_test, dSplit['y4_test'], 
                                                                                                          dPredict_train['predictions_train_y4_{}'.format(item)],
                                                                                                          dPredictProba_train['predictions_train_y4_{}'.format(item)],
                                                                                                          dPredict_test['predictions_test_y4_{}'.format(item)],
                                                                                                          dPredictProba_test['predictions_test_y4_{}'.format(item)])

for item2 in pp2Key:
    dThreat2_train['hasil_train_{}'.format(item2)], dThreat2_test['hasil_test_{}'.format(item2)]= calc_metrics(X_train, dSplit['y4_train'], X_test, dSplit['y4_test'], 
                                                                                                          dPredict2_train['predictions2_train_y4_{}'.format(item2)],
                                                                                                          dPredict2Proba_train['predictions2_train_y4_{}'.format(item2)],
                                                                                                          dPredict2_test['predictions2_test_y4_{}'.format(item2)],
                                                                                                          dPredict2Proba_test['predictions2_test_y4_{}'.format(item2)])

In [29]:
dThreat_train_key = list(dThreat_train.keys())
dThreat_test_key = list(dThreat_test.keys())
listItem = []

for train, test in zip(dThreat_train_key, dThreat_test_key):
    listItem.append(['xgb_{}'.format(test),
                     dObscene_train[train]['accuracy'],
                     dObscene_test[test]['accuracy'],
                     dObscene_train[train]['roc'],
                     dObscene_test[test]['roc'],
                     dObscene_train[train]['f1'],
                     dObscene_test[test]['f1'],
                     dObscene_train[train]['matthew'],
                     dObscene_test[test]['matthew'],
                     dObscene_train[train]['logloss'],
                     dObscene_test[test]['logloss'],])


dfThreatResult = pd.DataFrame(columns=['Model','Train Accuracy', 'Test Accuracy', 'Train ROC AUC', 'Test ROC AUC', 'Train F1 Score','Test F1 Score', 
                                      'Train Matthews Corr Coef', 'Test Matthew Corr Coef','Train Log Loss','Test Log Loss'], data=listItem)
dfThreatResult

,Model,Train Accuracy,Test Accuracy,Train ROC AUC,Test ROC AUC,Train F1 Score,Test F1 Score,Train Matthews Corr Coef,Test Matthew Corr Coef,Train Log Loss,Test Log Loss
0,xgb_hasil_test_pp_normal_ngram_1,0.968487,0.967455,0.913297,0.903606,0.785333,0.785118,0.622734,0.618162,0.113779,0.117484
1,xgb_hasil_test_pp_normal_ngram_2,0.968487,0.967455,0.913297,0.903606,0.785333,0.785118,0.622734,0.618162,0.113779,0.117484
2,xgb_hasil_test_pp_normal_ngram_3,0.968487,0.967455,0.913297,0.903606,0.785333,0.785118,0.622734,0.618162,0.113779,0.117484
3,xgb_hasil_test_pp_noCase_ngram_1,0.970036,0.969481,0.933293,0.926465,0.801750,0.804787,0.646052,0.647968,0.102852,0.106238
4,xgb_hasil_test_pp_noCase_ngram_2,0.970036,0.969481,0.933293,0.926465,0.801750,0.804787,0.646052,0.647968,0.102852,0.106238
5,xgb_hasil_test_pp_noCase_ngram_3,0.970036,0.969481,0.933293,0.926465,0.801750,0.804787,0.646052,0.647968,0.102852,0.106238
6,xgb_hasil_test_pp_noPunc_ngram_1,0.970107,0.968917,0.938751,0.933341,0.802483,0.800247,0.647114,0.640038,0.100235,0.104248
7,xgb_hasil_test_pp_noPunc_ngram_2,0.970107,0.968917,0.938751,0.933341,0.802483,0.800247,0.647114,0.640038,0.100235,0.104248
8,xgb_hasil_test_pp_noPunc_ngram_3,0.970107,0.968917,0.938751,0.933341,0.802483,0.800247,0.647114,0.640038,0.100235,0.104248
9,xgb_hasil_test_pp_noCasePunc_ngram_1,0.973124,0.972155,0.959854,0.953510,0.830194,0.829331,0.690167,0.685731,0.087135,0.090946


In [30]:
dThreat2_train_key = list(dThreat2_train.keys())
dThreat2_test_key = list(dThreat2_test.keys())
listItem = []

for train, test in zip(dThreat2_train_key, dThreat2_test_key):
    listItem.append(['xgb_{}'.format(test),
                     dObscene2_train[train]['accuracy'],
                     dObscene2_test[test]['accuracy'],
                     dObscene2_train[train]['roc'],
                     dObscene2_test[test]['roc'],
                     dObscene2_train[train]['f1'],
                     dObscene2_test[test]['f1'],
                     dObscene2_train[train]['matthew'],
                     dObscene2_test[test]['matthew'],
                     dObscene2_train[train]['logloss'],
                     dObscene2_test[test]['logloss'],])


dfThreat2Result = pd.DataFrame(columns=['Model','Train Accuracy', 'Test Accuracy', 'Train ROC AUC', 'Test ROC AUC', 'Train F1 Score','Test F1 Score', 
                                      'Train Matthews Corr Coef', 'Test Matthew Corr Coef','Train Log Loss','Test Log Loss'], data=listItem)
dfThreat2Result

,Model,Train Accuracy,Test Accuracy,Train ROC AUC,Test ROC AUC,Train F1 Score,Test F1 Score,Train Matthews Corr Coef,Test Matthew Corr Coef,Train Log Loss,Test Log Loss
0,xgb_hasil_test_pp2_normal_ngram_1,0.968621,0.967894,0.913189,0.903427,0.784573,0.787636,0.624567,0.624309,0.112214,0.116184
1,xgb_hasil_test_pp2_normal_ngram_2,0.968621,0.967894,0.913189,0.903427,0.784573,0.787636,0.624567,0.624309,0.112214,0.116184
2,xgb_hasil_test_pp2_normal_ngram_3,0.968621,0.967894,0.913189,0.903427,0.784573,0.787636,0.624567,0.624309,0.112214,0.116184
3,xgb_hasil_test_pp2_noCase_ngram_1,0.971056,0.970421,0.932047,0.925628,0.808878,0.812410,0.660442,0.661061,0.100504,0.103945
4,xgb_hasil_test_pp2_noCase_ngram_2,0.971056,0.970421,0.932047,0.925628,0.808878,0.812410,0.660442,0.661061,0.100504,0.103945
5,xgb_hasil_test_pp2_noCase_ngram_3,0.971056,0.970421,0.932047,0.925628,0.808878,0.812410,0.660442,0.661061,0.100504,0.103945
6,xgb_hasil_test_pp2_noPunc_ngram_1,0.971020,0.970108,0.937527,0.932185,0.808475,0.808983,0.659920,0.656476,0.097894,0.101891
7,xgb_hasil_test_pp2_noPunc_ngram_2,0.971020,0.970108,0.937527,0.932185,0.808475,0.808983,0.659920,0.656476,0.097894,0.101891
8,xgb_hasil_test_pp2_noPunc_ngram_3,0.971020,0.970108,0.937527,0.932185,0.808475,0.808983,0.659920,0.656476,0.097894,0.101891
9,xgb_hasil_test_pp2_noCasePunc_ngram_1,0.973581,0.972761,0.958718,0.952762,0.832014,0.832927,0.695926,0.693385,0.083853,0.087386


## Insult

In [31]:
dInsult_train = {}
dInsult_test = {}
dInsult2_train = {}
dInsult2_test = {}

for item in ppKey:
    dInsult_train['hasil_train_{}'.format(item)], dInsult_test['hasil_test_{}'.format(item)]= calc_metrics(X_train, dSplit['y5_train'], X_test, dSplit['y5_test'], 
                                                                                                          dPredict_train['predictions_train_y5_{}'.format(item)],
                                                                                                          dPredictProba_train['predictions_train_y5_{}'.format(item)],
                                                                                                          dPredict_test['predictions_test_y5_{}'.format(item)],
                                                                                                          dPredictProba_test['predictions_test_y5_{}'.format(item)])

for item2 in pp2Key:
    dInsult2_train['hasil_train_{}'.format(item2)], dInsult2_test['hasil_test_{}'.format(item2)]= calc_metrics(X_train, dSplit['y5_train'], X_test, dSplit['y5_test'], 
                                                                                                          dPredict2_train['predictions2_train_y5_{}'.format(item2)],
                                                                                                          dPredict2Proba_train['predictions2_train_y5_{}'.format(item2)],
                                                                                                          dPredict2_test['predictions2_test_y5_{}'.format(item2)],
                                                                                                          dPredict2Proba_test['predictions2_test_y5_{}'.format(item2)])

In [32]:
dInsult_train_key = list(dInsult_train.keys())
dInsult_test_key = list(dInsult_test.keys())
listItem = []

for train, test in zip(dInsult_train_key, dInsult_test_key):
    listItem.append(['xgb_{}'.format(test),
                     dInsult_train[train]['accuracy'],
                     dInsult_test[test]['accuracy'],
                     dInsult_train[train]['roc'],
                     dInsult_test[test]['roc'],
                     dInsult_train[train]['f1'],
                     dInsult_test[test]['f1'],
                     dInsult_train[train]['matthew'],
                     dInsult_test[test]['matthew'],
                     dInsult_train[train]['logloss'],
                     dInsult_test[test]['logloss'],])


dfInsultResult = pd.DataFrame(columns=['Model','Train Accuracy', 'Test Accuracy', 'Train ROC AUC', 'Test ROC AUC', 'Train F1 Score','Test F1 Score', 
                                      'Train Matthews Corr Coef', 'Test Matthew Corr Coef','Train Log Loss','Test Log Loss'], data=listItem)
dfInsultResult

,Model,Train Accuracy,Test Accuracy,Train ROC AUC,Test ROC AUC,Train F1 Score,Test F1 Score,Train Matthews Corr Coef,Test Matthew Corr Coef,Train Log Loss,Test Log Loss
0,xgb_hasil_test_pp_normal_ngram_1,0.960053,0.959559,0.901280,0.895972,0.665611,0.667539,0.428387,0.423485,0.126146,0.128723
1,xgb_hasil_test_pp_normal_ngram_2,0.960053,0.959559,0.901280,0.895972,0.665611,0.667539,0.428387,0.423485,0.126146,0.128723
2,xgb_hasil_test_pp_normal_ngram_3,0.960053,0.959559,0.901280,0.895972,0.665611,0.667539,0.428387,0.423485,0.126146,0.128723
3,xgb_hasil_test_pp_noCase_ngram_1,0.963357,0.963068,0.918833,0.915025,0.724698,0.727648,0.505308,0.505678,0.117536,0.119758
4,xgb_hasil_test_pp_noCase_ngram_2,0.963357,0.963068,0.918833,0.915025,0.724698,0.727648,0.505308,0.505678,0.117536,0.119758
5,xgb_hasil_test_pp_noCase_ngram_3,0.963357,0.963068,0.918833,0.915025,0.724698,0.727648,0.505308,0.505678,0.117536,0.119758
6,xgb_hasil_test_pp_noPunc_ngram_1,0.961853,0.961543,0.923583,0.920501,0.694530,0.696065,0.469232,0.467584,0.117248,0.118818
7,xgb_hasil_test_pp_noPunc_ngram_2,0.961853,0.961543,0.923583,0.920501,0.694530,0.696065,0.469232,0.467584,0.117248,0.118818
8,xgb_hasil_test_pp_noPunc_ngram_3,0.961853,0.961543,0.923583,0.920501,0.694530,0.696065,0.469232,0.467584,0.117248,0.118818
9,xgb_hasil_test_pp_noCasePunc_ngram_1,0.964476,0.964447,0.939416,0.934096,0.731458,0.736911,0.523122,0.527440,0.107411,0.108743


In [33]:
dInsult2_train_key = list(dInsult2_train.keys())
dInsult2_test_key = list(dInsult2_test.keys())
listItem = []

for train, test in zip(dInsult2_train_key, dInsult2_test_key):
    listItem.append(['xgb_{}'.format(test),
                     dInsult2_train[train]['accuracy'],
                     dInsult2_test[test]['accuracy'],
                     dInsult2_train[train]['roc'],
                     dInsult2_test[test]['roc'],
                     dInsult2_train[train]['f1'],
                     dInsult2_test[test]['f1'],
                     dInsult2_train[train]['matthew'],
                     dInsult2_test[test]['matthew'],
                     dInsult2_train[train]['logloss'],
                     dInsult2_test[test]['logloss'],])


dfInsult2Result = pd.DataFrame(columns=['Model','Train Accuracy', 'Test Accuracy', 'Train ROC AUC', 'Test ROC AUC', 'Train F1 Score','Test F1 Score', 
                                      'Train Matthews Corr Coef', 'Test Matthew Corr Coef','Train Log Loss','Test Log Loss'], data=listItem)
dfInsult2Result

,Model,Train Accuracy,Test Accuracy,Train ROC AUC,Test ROC AUC,Train F1 Score,Test F1 Score,Train Matthews Corr Coef,Test Matthew Corr Coef,Train Log Loss,Test Log Loss
0,xgb_hasil_test_pp2_normal_ngram_1,0.960743,0.960039,0.906404,0.899135,0.673174,0.673277,0.443248,0.433917,0.124345,0.127389
1,xgb_hasil_test_pp2_normal_ngram_2,0.960743,0.960039,0.906404,0.899135,0.673174,0.673277,0.443248,0.433917,0.124345,0.127389
2,xgb_hasil_test_pp2_normal_ngram_3,0.960743,0.960039,0.906404,0.899135,0.673174,0.673277,0.443248,0.433917,0.124345,0.127389
3,xgb_hasil_test_pp2_noCase_ngram_1,0.964109,0.963256,0.920618,0.916015,0.731492,0.731114,0.518474,0.510198,0.115063,0.117957
4,xgb_hasil_test_pp2_noCase_ngram_2,0.964109,0.963256,0.920618,0.916015,0.731492,0.731114,0.518474,0.510198,0.115063,0.117957
5,xgb_hasil_test_pp2_noCase_ngram_3,0.964109,0.963256,0.920618,0.916015,0.731492,0.731114,0.518474,0.510198,0.115063,0.117957
6,xgb_hasil_test_pp2_noPunc_ngram_1,0.962435,0.962066,0.925459,0.921984,0.699472,0.702122,0.480303,0.478049,0.114655,0.116612
7,xgb_hasil_test_pp2_noPunc_ngram_2,0.962435,0.962066,0.925459,0.921984,0.699472,0.702122,0.480303,0.478049,0.114655,0.116612
8,xgb_hasil_test_pp2_noPunc_ngram_3,0.962435,0.962066,0.925459,0.921984,0.699472,0.702122,0.480303,0.478049,0.114655,0.116612
9,xgb_hasil_test_pp2_noCasePunc_ngram_1,0.965479,0.965408,0.942604,0.939595,0.740672,0.747361,0.540650,0.544802,0.103940,0.105673


## Identity Hate

In [34]:
dIdHate_train = {}
dIdHate_test = {}
dIdHate2_train = {}
dIdHate2_test = {}

for item in ppKey:
    dIdHate_train['hasil_train_{}'.format(item)], dIdHate_test['hasil_test_{}'.format(item)]= calc_metrics(X_train, dSplit['y6_train'], X_test, dSplit['y6_test'], 
                                                                                                          dPredict_train['predictions_train_y6_{}'.format(item)],
                                                                                                          dPredictProba_train['predictions_train_y6_{}'.format(item)],
                                                                                                          dPredict_test['predictions_test_y6_{}'.format(item)],
                                                                                                          dPredictProba_test['predictions_test_y6_{}'.format(item)])

for item2 in pp2Key:
    dIdHate2_train['hasil_train_{}'.format(item2)], dIdHate2_test['hasil_test_{}'.format(item2)]= calc_metrics(X_train, dSplit['y6_train'], X_test, dSplit['y6_test'], 
                                                                                                          dPredict2_train['predictions2_train_y6_{}'.format(item2)],
                                                                                                          dPredict2Proba_train['predictions2_train_y6_{}'.format(item2)],
                                                                                                          dPredict2_test['predictions2_test_y6_{}'.format(item2)],
                                                                                                          dPredict2Proba_test['predictions2_test_y6_{}'.format(item2)])

In [35]:
dIdHate_train_key = list(dIdHate_train.keys())
dIdHate_test_key = list(dIdHate_test.keys())
listItem = []

for train, test in zip(dIdHate_train_key, dIdHate_test_key):
    listItem.append(['xgb_{}'.format(test),
                     dIdHate_train[train]['accuracy'],
                     dIdHate_test[test]['accuracy'],
                     dIdHate_train[train]['roc'],
                     dIdHate_test[test]['roc'],
                     dIdHate_train[train]['f1'],
                     dIdHate_test[test]['f1'],
                     dIdHate_train[train]['matthew'],
                     dIdHate_test[test]['matthew'],
                     dIdHate_train[train]['logloss'],
                     dIdHate_test[test]['logloss'],])


dfIdHateResult = pd.DataFrame(columns=['Model','Train Accuracy', 'Test Accuracy', 'Train ROC AUC', 'Test ROC AUC', 'Train F1 Score','Test F1 Score', 
                                      'Train Matthews Corr Coef', 'Test Matthew Corr Coef','Train Log Loss','Test Log Loss'], data=listItem)
dfIdHateResult

,Model,Train Accuracy,Test Accuracy,Train ROC AUC,Test ROC AUC,Train F1 Score,Test F1 Score,Train Matthews Corr Coef,Test Matthew Corr Coef,Train Log Loss,Test Log Loss
0,xgb_hasil_test_pp_normal_ngram_1,0.991862,0.991895,0.908335,0.898541,0.587386,0.578051,0.295037,0.251566,0.034702,0.034353
1,xgb_hasil_test_pp_normal_ngram_2,0.991862,0.991895,0.908335,0.898541,0.587386,0.578051,0.295037,0.251566,0.034702,0.034353
2,xgb_hasil_test_pp_normal_ngram_3,0.991862,0.991895,0.908335,0.898541,0.587386,0.578051,0.295037,0.251566,0.034702,0.034353
3,xgb_hasil_test_pp_noCase_ngram_1,0.992023,0.992167,0.932952,0.921355,0.611612,0.609833,0.328542,0.310086,0.032027,0.032158
4,xgb_hasil_test_pp_noCase_ngram_2,0.992023,0.992167,0.932952,0.921355,0.611612,0.609833,0.328542,0.310086,0.032027,0.032158
5,xgb_hasil_test_pp_noCase_ngram_3,0.992023,0.992167,0.932952,0.921355,0.611612,0.609833,0.328542,0.310086,0.032027,0.032158
6,xgb_hasil_test_pp_noPunc_ngram_1,0.991952,0.991895,0.936641,0.920953,0.600542,0.598786,0.313635,0.275298,0.032110,0.031800
7,xgb_hasil_test_pp_noPunc_ngram_2,0.991952,0.991895,0.936641,0.920953,0.600542,0.598786,0.313635,0.275298,0.032110,0.031800
8,xgb_hasil_test_pp_noPunc_ngram_3,0.991952,0.991895,0.936641,0.920953,0.600542,0.598786,0.313635,0.275298,0.032110,0.031800
9,xgb_hasil_test_pp_noCasePunc_ngram_1,0.992310,0.992271,0.954953,0.940159,0.638653,0.632445,0.374615,0.340410,0.029105,0.029345


In [36]:
dIdHate2_train_key = list(dIdHate2_train.keys())
dIdHate2_test_key = list(dIdHate2_test.keys())
listItem = []

for train, test in zip(dIdHate2_train_key, dIdHate2_test_key):
    listItem.append(['xgb_{}'.format(test),
                     dIdHate2_train[train]['accuracy'],
                     dIdHate2_test[test]['accuracy'],
                     dIdHate2_train[train]['roc'],
                     dIdHate2_test[test]['roc'],
                     dIdHate2_train[train]['f1'],
                     dIdHate2_test[test]['f1'],
                     dIdHate2_train[train]['matthew'],
                     dIdHate2_test[test]['matthew'],
                     dIdHate2_train[train]['logloss'],
                     dIdHate2_test[test]['logloss'],])


dfIdHate2Result = pd.DataFrame(columns=['Model','Train Accuracy', 'Test Accuracy', 'Train ROC AUC', 'Test ROC AUC', 'Train F1 Score','Test F1 Score', 
                                      'Train Matthews Corr Coef', 'Test Matthew Corr Coef','Train Log Loss','Test Log Loss'], data=listItem)
dfIdHate2Result

,Model,Train Accuracy,Test Accuracy,Train ROC AUC,Test ROC AUC,Train F1 Score,Test F1 Score,Train Matthews Corr Coef,Test Matthew Corr Coef,Train Log Loss,Test Log Loss
0,xgb_hasil_test_pp2_normal_ngram_1,0.991925,0.991874,0.910906,0.891094,0.588715,0.577872,0.305062,0.249126,0.033937,0.034267
1,xgb_hasil_test_pp2_normal_ngram_2,0.991925,0.991874,0.910906,0.891094,0.588715,0.577872,0.305062,0.249126,0.033937,0.034267
2,xgb_hasil_test_pp2_normal_ngram_3,0.991925,0.991874,0.910906,0.891094,0.588715,0.577872,0.305062,0.249126,0.033937,0.034267
3,xgb_hasil_test_pp2_noCase_ngram_1,0.992175,0.991979,0.934491,0.918649,0.619351,0.602923,0.350231,0.286686,0.031136,0.031691
4,xgb_hasil_test_pp2_noCase_ngram_2,0.992175,0.991979,0.934491,0.918649,0.619351,0.602923,0.350231,0.286686,0.031136,0.031691
5,xgb_hasil_test_pp2_noCase_ngram_3,0.992175,0.991979,0.934491,0.918649,0.619351,0.602923,0.350231,0.286686,0.031136,0.031691
6,xgb_hasil_test_pp2_noPunc_ngram_1,0.992184,0.992125,0.934599,0.914958,0.616146,0.609361,0.350155,0.305832,0.031092,0.031300
7,xgb_hasil_test_pp2_noPunc_ngram_2,0.992184,0.992125,0.934599,0.914958,0.616146,0.609361,0.350155,0.305832,0.031092,0.031300
8,xgb_hasil_test_pp2_noPunc_ngram_3,0.992184,0.992125,0.934599,0.914958,0.616146,0.609361,0.350155,0.305832,0.031092,0.031300
9,xgb_hasil_test_pp2_noCasePunc_ngram_1,0.992569,0.992459,0.954747,0.944032,0.652299,0.644876,0.407772,0.366266,0.027776,0.028356


## Clean

In [37]:
dClean_train = {}
dClean_test = {}
dClean2_train = {}
dClean2_test = {}

for item in ppKey:
    dClean_train['hasil_train_{}'.format(item)], dClean_test['hasil_test_{}'.format(item)]= calc_metrics(X_train, dSplit['y7_train'], X_test, dSplit['y7_test'], 
                                                                                                          dPredict_train['predictions_train_y7_{}'.format(item)],
                                                                                                          dPredictProba_train['predictions_train_y7_{}'.format(item)],
                                                                                                          dPredict_test['predictions_test_y7_{}'.format(item)],
                                                                                                          dPredictProba_test['predictions_test_y7_{}'.format(item)])

for item2 in pp2Key:
    dClean2_train['hasil_train_{}'.format(item2)], dClean2_test['hasil_test_{}'.format(item2)]= calc_metrics(X_train, dSplit['y7_train'], X_test, dSplit['y7_test'], 
                                                                                                          dPredict2_train['predictions2_train_y7_{}'.format(item2)],
                                                                                                          dPredict2Proba_train['predictions2_train_y7_{}'.format(item2)],
                                                                                                          dPredict2_test['predictions2_test_y7_{}'.format(item2)],
                                                                                                          dPredict2Proba_test['predictions2_test_y7_{}'.format(item2)])

In [38]:
dClean_train_key = list(dClean_train.keys())
dClean_test_key = list(dClean_test.keys())
listItem = []

for train, test in zip(dClean_train_key, dClean_test_key):
    listItem.append(['xgb_{}'.format(test),
                     dClean_train[train]['accuracy'],
                     dClean_test[test]['accuracy'],
                     dClean_train[train]['roc'],
                     dClean_test[test]['roc'],
                     dClean_train[train]['f1'],
                     dClean_test[test]['f1'],
                     dClean_train[train]['matthew'],
                     dClean_test[test]['matthew'],
                     dClean_train[train]['logloss'],
                     dClean_test[test]['logloss'],])


dfCleanResult = pd.DataFrame(columns=['Model','Train Accuracy', 'Test Accuracy', 'Train ROC AUC', 'Test ROC AUC', 'Train F1 Score','Test F1 Score', 
                                      'Train Matthews Corr Coef', 'Test Matthew Corr Coef','Train Log Loss','Test Log Loss'], data=listItem)
dfCleanResult

,Model,Train Accuracy,Test Accuracy,Train ROC AUC,Test ROC AUC,Train F1 Score,Test F1 Score,Train Matthews Corr Coef,Test Matthew Corr Coef,Train Log Loss,Test Log Loss
0,xgb_hasil_test_pp_normal_ngram_1,0.925201,0.925343,0.862283,0.857854,0.691364,0.696460,0.492220,0.496201,0.229334,0.230757
1,xgb_hasil_test_pp_normal_ngram_2,0.925201,0.925343,0.862283,0.857854,0.691364,0.696460,0.492220,0.496201,0.229334,0.230757
2,xgb_hasil_test_pp_normal_ngram_3,0.925201,0.925343,0.862283,0.857854,0.691364,0.696460,0.492220,0.496201,0.229334,0.230757
3,xgb_hasil_test_pp_noCase_ngram_1,0.929059,0.928915,0.881242,0.875790,0.716117,0.718724,0.527503,0.528663,0.215536,0.216763
4,xgb_hasil_test_pp_noCase_ngram_2,0.929059,0.928915,0.881242,0.875790,0.716117,0.718724,0.527503,0.528663,0.215536,0.216763
5,xgb_hasil_test_pp_noCase_ngram_3,0.929059,0.928915,0.881242,0.875790,0.716117,0.718724,0.527503,0.528663,0.215536,0.216763
6,xgb_hasil_test_pp_noPunc_ngram_1,0.929283,0.929332,0.889222,0.885756,0.717696,0.721015,0.529447,0.532384,0.213696,0.214833
7,xgb_hasil_test_pp_noPunc_ngram_2,0.929283,0.929332,0.889222,0.885756,0.717696,0.721015,0.529447,0.532384,0.213696,0.214833
8,xgb_hasil_test_pp_noPunc_ngram_3,0.929283,0.929332,0.889222,0.885756,0.717696,0.721015,0.529447,0.532384,0.213696,0.214833
9,xgb_hasil_test_pp_noCasePunc_ngram_1,0.935067,0.935265,0.905359,0.903093,0.752005,0.755805,0.578694,0.582443,0.197688,0.198267


In [39]:
dClean2_train_key = list(dClean2_train.keys())
dClean2_test_key = list(dClean2_test.keys())
listItem = []

for train, test in zip(dClean2_train_key, dClean2_test_key):
    listItem.append(['xgb_{}'.format(test),
                     dClean2_train[train]['accuracy'],
                     dClean2_test[test]['accuracy'],
                     dClean2_train[train]['roc'],
                     dClean2_test[test]['roc'],
                     dClean2_train[train]['f1'],
                     dClean2_test[test]['f1'],
                     dClean2_train[train]['matthew'],
                     dClean2_test[test]['matthew'],
                     dClean2_train[train]['logloss'],
                     dClean2_test[test]['logloss'],])


dfClean2Result = pd.DataFrame(columns=['Model','Train Accuracy', 'Test Accuracy', 'Train ROC AUC', 'Test ROC AUC', 'Train F1 Score','Test F1 Score', 
                                      'Train Matthews Corr Coef', 'Test Matthew Corr Coef','Train Log Loss','Test Log Loss'], data=listItem)
dfClean2Result

,Model,Train Accuracy,Test Accuracy,Train ROC AUC,Test ROC AUC,Train F1 Score,Test F1 Score,Train Matthews Corr Coef,Test Matthew Corr Coef,Train Log Loss,Test Log Loss
0,xgb_hasil_test_pp2_normal_ngram_1,0.925102,0.925343,0.864442,0.858758,0.690180,0.696460,0.491424,0.496201,0.227706,0.229170
1,xgb_hasil_test_pp2_normal_ngram_2,0.925102,0.925343,0.864442,0.858758,0.690180,0.696460,0.491424,0.496201,0.227706,0.229170
2,xgb_hasil_test_pp2_normal_ngram_3,0.925102,0.925343,0.864442,0.858758,0.690180,0.696460,0.491424,0.496201,0.227706,0.229170
3,xgb_hasil_test_pp2_noCase_ngram_1,0.929820,0.930022,0.881542,0.877269,0.719354,0.724951,0.534569,0.538446,0.213066,0.213952
4,xgb_hasil_test_pp2_noCase_ngram_2,0.929820,0.930022,0.881542,0.877269,0.719354,0.724951,0.534569,0.538446,0.213066,0.213952
5,xgb_hasil_test_pp2_noCase_ngram_3,0.929820,0.930022,0.881542,0.877269,0.719354,0.724951,0.534569,0.538446,0.213066,0.213952
6,xgb_hasil_test_pp2_noPunc_ngram_1,0.930277,0.930314,0.890417,0.886109,0.722573,0.726488,0.538455,0.541022,0.211153,0.212297
7,xgb_hasil_test_pp2_noPunc_ngram_2,0.930277,0.930314,0.890417,0.886109,0.722573,0.726488,0.538455,0.541022,0.211153,0.212297
8,xgb_hasil_test_pp2_noPunc_ngram_3,0.930277,0.930314,0.890417,0.886109,0.722573,0.726488,0.538455,0.541022,0.211153,0.212297
9,xgb_hasil_test_pp2_noCasePunc_ngram_1,0.936320,0.936456,0.907019,0.903570,0.757390,0.761553,0.589230,0.592175,0.193332,0.193925


In [40]:
import pickle

In [43]:
dfToxicResult.to_csv('xgb_toxic.csv',index=False)
dfToxic2Result.to_csv('xgb_toxic2.csv',index=False)

dfSToxicResult.to_csv('xgb_stoxic.csv',index=False)
dfSToxic2Result.to_csv('xgb_stoxic2.csv',index=False)

dfObsceneResult.to_csv('xgb_obscene.csv',index=False)
dfObscene2Result.to_csv('xgb_obscene2.csv',index=False)

dfThreatResult.to_csv('xgb_threat.csv',index=False)
dfThreat2Result.to_csv('xgb_threat2.csv',index=False)

dfInsultResult.to_csv('xgb_insult.csv',index=False)
dfInsult2Result.to_csv('xgb_insult2.csv',index=False)

dfIdHateResult.to_csv('xgb_idHate.csv',index=False)
dfIdHate2Result.to_csv('xgb_idHate2.csv',index=False)

dfCleanResult.to_csv('xgb_clean.csv',index=False)
dfClean2Result.to_csv('xgb_clean2.csv',index=False)

In [42]:
model = '_xgb'

pickle.dump(dToxic_train, open('dToxic_train{}.sav'.format(model), 'wb'))
pickle.dump(dToxic_test, open('dToxic_test{}.sav'.format(model), 'wb'))
pickle.dump(dToxic2_train, open('dToxic2_train{}.sav'.format(model), 'wb'))
pickle.dump(dToxic2_test, open('dToxic2_test{}.sav'.format(model), 'wb'))

pickle.dump(dSToxic_train, open('dSToxic_train{}.sav'.format(model), 'wb'))
pickle.dump(dSToxic_test, open('dSToxic_test{}.sav'.format(model), 'wb'))
pickle.dump(dSToxic2_train, open('dSToxic2_train{}.sav'.format(model), 'wb'))
pickle.dump(dSToxic2_test, open('dSToxic2_test{}.sav'.format(model), 'wb'))

pickle.dump(dObscene_train, open('dObscene_train{}.sav'.format(model), 'wb'))
pickle.dump(dObscene_test, open('dObscene_test{}.sav'.format(model), 'wb'))
pickle.dump(dObscene2_train, open('dObscene2_train{}.sav'.format(model), 'wb'))
pickle.dump(dObscene2_test, open('dObscene2_test{}.sav'.format(model), 'wb'))

pickle.dump(dThreat_train, open('dThreat_train{}.sav'.format(model), 'wb'))
pickle.dump(dThreat_test, open('dThreat_test{}.sav'.format(model), 'wb'))
pickle.dump(dThreat2_train, open('dThreat2_train{}.sav'.format(model), 'wb'))
pickle.dump(dThreat2_test, open('dThreat2_test{}.sav'.format(model), 'wb'))

pickle.dump(dInsult_train, open('dInsult_train{}.sav'.format(model), 'wb'))
pickle.dump(dInsult_test, open('dInsult_test{}.sav'.format(model), 'wb'))
pickle.dump(dInsult2_train, open('dInsult2_train{}.sav'.format(model), 'wb'))
pickle.dump(dInsult2_test, open('dInsult2_test{}.sav'.format(model), 'wb'))

pickle.dump(dIdHate_train, open('dIdHate_train{}.sav'.format(model), 'wb'))
pickle.dump(dIdHate_test, open('dIdHate_test{}.sav'.format(model), 'wb'))
pickle.dump(dIdHate2_train, open('dIdHate2_train{}.sav'.format(model), 'wb'))
pickle.dump(dIdHate2_test, open('dIdHate2_test{}.sav'.format(model), 'wb'))

pickle.dump(dClean_train, open('dClean_train{}.sav'.format(model), 'wb'))
pickle.dump(dClean_test, open('dClean_test{}.sav'.format(model), 'wb'))
pickle.dump(dClean2_train, open('dClean2_train{}.sav'.format(model), 'wb'))
pickle.dump(dClean2_test, open('dClean2_test{}.sav'.format(model), 'wb'))

In [44]:
pickle.dump(dPredict_train, open('dPredict_train.sav', 'wb'))
pickle.dump(dPredict_test, open('dPredict_test.sav', 'wb'))
pickle.dump(dPredictProba_train, open('dPredictProba_train.sav', 'wb'))
pickle.dump(dPredictProba_test, open('dPredictProba_test.sav', 'wb'))

pickle.dump(dPredict2_train, open('dPredict2_train.sav', 'wb'))
pickle.dump(dPredict2_test, open('dPredict2_test.sav', 'wb'))
pickle.dump(dPredict2Proba_train, open('dPredict2Proba_train.sav', 'wb'))
pickle.dump(dPredict2Proba_test, open('dPredict2Proba_test.sav', 'wb'))